In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
import pickle

In [18]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
#preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [20]:
#encoding categorical variables
label_enconder_gender=LabelEncoder()
data['Gender']=label_enconder_gender.fit_transform(data['Gender'])


In [21]:
#encoding geography column
onehot_encoder_geography=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))


In [22]:
#combine one hot encoded columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [23]:
#split the data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']


In [24]:
#split the data in training and testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [25]:
#scaling the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [26]:
#save the encoders and scaler for future use
with open('label_encoder_gender', 'wb') as file:
    pickle.dump(label_enconder_gender, file)
with open('onehot_encoder_geography', 'wb') as file:
    pickle.dump(onehot_encoder_geography, file)
with open('scaler', 'wb') as file:
    pickle.dump(scaler, file)

ANN REgression problem statement

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [36]:
#build the ANN model
model=Sequential([    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])
#compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

c:\Users\Syed Sheezan\AI-PROJECTS\ANN-classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
#set my tensorboard
from tensorflow.keras.callbacks import TensorBoard
import datetime

log_dir='regression_logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
TensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [38]:
#set up early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

In [39]:
#train the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[TensorBoard_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 100377.8594 - mae: 100377.8594 - val_loss: 98517.9297 - val_mae: 98517.9297
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 99642.9766 - mae: 99642.9766 - val_loss: 97037.6641 - val_mae: 97037.6641
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 97075.4766 - mae: 97075.4766 - val_loss: 93274.8516 - val_mae: 93274.8516
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 92007.1172 - mae: 92007.1172 - val_loss: 86966.7891 - val_mae: 86966.7891
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 84583.2031 - mae: 84583.2031 - val_loss: 78804.5078 - val_mae: 78804.5078
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 75788.7578 - mae: 75788.7578 - val_loss: 70187.6172 - val_mae: 70187.6172
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 67139.2656 - mae: 67139.2656 - val_loss: 62438.8633 - val_mae: 62438.8633
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 24m

In [40]:
#launch tensorboard extension
%load_ext tensorboard



In [42]:
%tensorboard --logdir=regression_logs/fit

Reusing TensorBoard on port 6006 (pid 11844), started 0:01:08 ago. (Use '!kill 11844' to kill it.)

In [43]:
#evaluate the model on test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 50303.4883 - mae: 50303.4883
Test MAE: 50303.48828125


In [44]:
#save model
model.save('salary_regression_model.h5')